# Welcome to :snowflake: Snowflake Notebooks :notebook:

Take your data analysis to the next level by working with Python and SQL seamlessly in [Snowflake Notebooks](https://docs.snowflake.com/LIMITEDACCESS/snowsight-notebooks/ui-snowsight-notebooks-about)! ⚡️

Here is a quick notebook to get you started on your first project! 🚀

## Adding Python Packages 🎒

Notebooks comes pre-installed with common Python libraries for data science 🧪 and machine learning 🧠, such as numpy, pandas, matplotlib, and more! 

If you are looking to use other packages, click on the `Packages` dropdown on the top right to add additional packages to your notebook.

For the purpose of this demo, let's add the `matplotlib` and `scipy` package from the package picker.

In [ ]:
# Import Python packages used in this notebook
import streamlit as st
import altair as alt

# Pre-installed libraries that comes with the notebook
import pandas as pd
import numpy as np

# Package that we just added
import matplotlib.pyplot as plt

## SQL Querying at your fingertips 💡 

We can easily switch between Python and SQL in the same worksheet. 

Let's write some SQL to generate sample data to play with. 

In [ ]:
-- Generating a synthetic dataset of Snowboard products, along with their price and rating
SELECT CONCAT('SNOW-',UNIFORM(1000,9999, RANDOM())) AS PRODUCT_ID, 
       ABS(NORMAL(5, 3, RANDOM())) AS RATING, 
       ABS(NORMAL(750, 200::FLOAT, RANDOM())) AS PRICE
FROM TABLE(GENERATOR(ROWCOUNT => 100));

## Back to Working in Python 🐍

You can give cells a name and refer to its output in subsequent cells.

We can access the SQL results directly in Python and convert the results to a pandas dataframe. 🐼

```python
# Access the SQL cell output as a Snowpark dataframe
my_snowpark_df = cell5.to_df()
``` 

```python
# Convert a SQL cell output into a pandas dataframe
my_df = cell5.to_pandas()
``` 

In [ ]:
df = cell5.to_pandas()
df

## 📊 Visualize your data

We can use [Altair](https://altair-viz.github.io/) to easily visualize our data distribution as a histogram.

In [ ]:
# Let's plot the results with Altair
chart = alt.Chart(df,title="Rating Distribution").mark_bar().encode(
    alt.X("RATING", bin=alt.Bin(step=2)),
    y='count()',
)

st.altair_chart(chart)

Let's say that you want to customize your chart and plot the kernel density estimate (KDE) and median. We can use matplotlib to plot the price distribution. Note that the `.plot` command uses `scipy` under the hood to compute the KDE profile, which we added as a package earlier in this tutorial.

In [ ]:
fig, ax = plt.subplots(figsize = (6,3))
plt.tick_params(left = False, right = False , labelleft = False) 

price = df["PRICE"]
price.plot(kind = "hist", density = True, bins = 15)
price.plot(kind="kde", color='#c44e52')


# Calculate percentiles
median = price.median()
ax.axvline(median,0, color='#dd8452', ls='--')
ax.text(median,0.8, f'Median: {median:.2f}  ',
        ha='right', va='center', color='#dd8452', transform=ax.get_xaxis_transform())

# Make our chart pretty
plt.style.use("bmh")
plt.title("Price Distribution")
plt.xlabel("PRICE (binned)")
left, right = plt.xlim()   
plt.xlim((0, right))  
# Remove ticks and spines
ax.tick_params(left = False, bottom = False)
for ax, spine in ax.spines.items():
    spine.set_visible(False)

plt.show()

## Working with data using Snowpark 🛠️

In addition to using your favorite Python data science libraries, you can also use the [Snowpark API](https://docs.snowflake.com/en/developer-guide/snowpark/index) to query and process your data at scale within the Notebook. 

First, you can get your session variable directly through the active notebook session. The session variable is the entrypoint that gives you access to using Snowflake's Python API.

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
session

In [ ]:
session.sql("SHOW TABLES in DATABASE SANDBOX limit 5")

In [ ]:
SHOW TABLES in DATABASE SANDBOX limit 5

In [ ]:
# session.use_schema("NOTEBOOK_DEMOS")
session.use_schema("PUBLIC")
session

For example, we can use Snowpark to save our pandas dataframe back to a table in Snowflake. 💾

In [ ]:
session.write_pandas(df,"SNOW_CATALOG",auto_create_table=True, table_type="temp")

Now that the `SNOW_CATALOG` table has been created, we can load the table using the following syntax: 

```python
df = session.table("<DATABASE_NAME>.<SCHEMA_NAME>.<TABLE_NAME>")
```

If your session is already set to the database and schema for the table you want to access, then you can reference the table name directly.

In [ ]:
df = session.table("SNOW_CATALOG")

Once we have loaded the table, we can call Snowpark's [`describe`](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrame.describe) to compute basic descriptive statistics. 

In [ ]:
df.describe()

In [ ]:
from snowflake.snowpark.functions import col
df.group_by("PRODUCT_ID").mean("RATING").rename("AVG(RATING)","AVG_RATING").filter(col("AVG_RATING")>10).explain()

## Using Python variables in SQL cells 🔖

You can use the Jinja syntax `{{..}}` to refer to Python variables within your SQL queries as follows. 

```python
threshold = 5
```

```sql
-- Reference Python variable in SQL
SELECT * FROM SNOW_CATALOG where RATING > {{threshold}}
```

Let's put this in practice to generate a distribution of values for ratings based on the mean and standard deviation values we set with Python.

In [ ]:
mean = 5 
stdev = 3

In [ ]:
-- Note how we use the Python variables `mean` and `stdev` to populate the SQL query
-- Note how the Python variables dynamically populate the SQL query
CREATE OR REPLACE TABLE SNOW_CATALOG AS 
SELECT CONCAT('SNOW-',UNIFORM(1000,9999, RANDOM())) AS PRODUCT_ID, 
       ABS(NORMAL({{mean}}, {{stdev}}, RANDOM())) AS RATING, 
       ABS(NORMAL(750, 200::FLOAT, RANDOM())) AS PRICE
FROM TABLE(GENERATOR(ROWCOUNT => 100));

In [ ]:
SELECT * FROM SNOW_CATALOG;

### Level up your subquery game! 🧑‍🎓

You can simplify long subqueries with [CTEs](https://docs.snowflake.com/en/user-guide/queries-cte) by combining what we've learned with Python and SQL cell result referencing. 

For example, if we want to compute the average rating of all products with ratings above 5. We would typically have to write something like the following:


In [ ]:
WITH RatingsAboveFive AS (
    SELECT RATING
    FROM SNOW_CATALOG
    WHERE RATING > 5
)
SELECT AVG(RATING) AS AVG_RATING_ABOVE_FIVE
FROM RatingsAboveFive;

With Snowflake Notebooks, the query is much simpler! You can get the same result by filtering a SQL table from another SQL cell by referencing it with Jinja, e.g., `{{my_cell}}`. 

In [ ]:
SELECT AVG(RATING) FROM {{cell23}}
WHERE RATING > 5

## Creating an interactive app with Streamlit 🪄

Putting this all together, let's build a Streamlit app to explore how different parameters impacts the shape of the data distribution histogram.

In [ ]:
from snowflake.snowpark.functions import min, max, mean
df.select(mean(col("RATING"))).collect()[0][0]

In [ ]:
df_pd = df.to_pandas()

In [ ]:
plt.scatter

In [ ]:
import streamlit as st
st.markdown("# Move the slider to adjust and watch the results update! 👇")

rating = st.slider('RATING FILTER', df_pd.RATING.min(),
                                        df_pd.RATING.max(),
                                        df_pd.RATING.mean()) 
price = st.slider('PRICE FILTER', df_pd.PRICE.min(),
                                      df_pd.PRICE.max(),
                                      df_pd.PRICE.mean()) 


result = df_pd[(df_pd.RATING>rating) & (df_pd.PRICE >price)]

fig, ax = plt.subplots(figsize=(3,2))
ax.scatter(x=result.RATING, y=result.PRICE, s = 3,color='lightblue', edgecolors='black', linewidths=0.2)
ax.set_xlabel("RATING", fontsize=3)
ax.set_ylabel("PRICE", fontsize=3)
ax.tick_params(axis='both', labelsize=3)

st.pyplot(fig)

## Run Faster with Keyboard Shortcuts 🏃

These shortcuts can help you navigate around your notebook more quickly. 

| Command | Shortcut |
| --- | ----------- |
| **Run this cell and advance** | SHIFT + ENTER |
| **Run this cell only** | CMD + ENTER |
| **Run all cells** | CMD + SHIFT + ENTER |
| **Add cell BELOW** | b |
| **Add cell ABOVE** | a |
| **Delete this cell** | d+d |

\
You can view the full list of shortcuts by clicking the `?` button on the bottom right.

In [ ]:
SELECT * FROM TPCXAI_SF0001_QUICKSTART.TRAINING.ORDERS limit 10;

In [ ]:
SELECT * FROM TPCXAI_SF0001_QUICKSTART.TRAINING.LINEITEM;
-- SELECT AVG(PRICE) AS AVG_PRICE, AVG(QUANTITY) as AVG_QUANTITY FROM TPCXAI_SF0001_QUICKSTART.TRAINING.LINEITEM GROUP BY QUANTITY ORDER BY AVG_PRICE limit 100;

In [ ]:
SELECT
    store,
    o_order_id,
    order_ts,
    SUM(o_customer_sk) OVER (PARTITION BY store ORDER BY order_ts ROWS BETWEEN 30 PRECEDING AND CURRENT ROW) AS rolling_7_orders
FROM
    TPCXAI_SF0001_QUICKSTART.TRAINING.ORDERS
-- WHERE
--     STORE > 5
ORDER BY 
    ROLLING_7_ORDERS

In [ ]:
SELECT
    STORE,
    COUNT(li.li_order_id) AS total_orders,
    SUM(li.price) AS total_amount
FROM
    TPCXAI_SF0001_QUICKSTART.TRAINING.LINEITEM li
JOIN
    TPCXAI_SF0001_QUICKSTART.TRAINING.ORDERS o ON o.o_order_id = li.li_order_id

WHERE
    o.ORDER_DATE BETWEEN '2021-01-01' AND '2021-10-01'
GROUP BY
    o.STORE
HAVING
    count(li.li_order_id)>59000
ORDER BY
    total_amount DESC
LIMIT 100;

## Keep Exploring Notebooks! 🧭

Check out our [sample notebook gallery](https://github.com/Snowflake-Labs/notebook-demo) and [documentation](https://docs.snowflake.com/LIMITEDACCESS/snowsight-notebooks/ui-snowsight-notebooks-about) to learn more!